In [ ]:
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data
import numpy as np
from matplotlib import pyplot as plt
import matplotlib.gridspec as gridspec

Utils

In [ ]:
def xavier_init(size):
    # Weight initializer
    in_dim = size[0]
    xavier_stddev = 1. / tf.sqrt(in_dim / 2.)
    return tf.random_normal(shape=size, stddev=xavier_stddev)


def plot(samples):
    # 3x3 plot for show generative net's output
    fig = plt.figure(figsize=(3, 3))
    gs = gridspec.GridSpec(3, 3)
    gs.update(wspace=0.05, hspace=0.05)

    for i, sample in enumerate(samples):
        ax = plt.subplot(gs[i])
        plt.axis('off')
        ax.set_xticklabels([])
        ax.set_yticklabels([])
        ax.set_aspect('equal')
        plt.imshow(sample.reshape(28, 28), cmap='Greys_r')

    return fig

# Building networks

Discriminative net 

In [ ]:
mnist_shape = 784
hidden_size = 256
output_shape = 10

input_D_ph = tf.placeholder(tf.float32, [None, mnist_shape], name='Real_X')

D_W1 = tf.Variable(xavier_init((mnist_shape, hidden_size)), name='D_W1')
D_b1 = tf.Variable(tf.zeros(hidden_size), name='D_b1')

D_W2 = tf.Variable(xavier_init((hidden_size, output_shape)), name='D_W2')
D_b2 = tf.Variable(tf.zeros(output_shape), name='D_b2')


def discriminative_network(x):
    layer1 = tf.matmul(x, D_W1) + D_b1
    layer1 = tf.nn.relu(layer1)
    
    layer2 = tf.matmul(layer1, D_W2) + D_b2
    layer2 = tf.nn.sigmoid(layer2)

    return layer2

Generative net

In [ ]:
z_dim = 300
input_G_ph = tf.placeholder(tf.float32, [None, z_dim], name='Fake_X')

G_W1 = tf.Variable(xavier_init((z_dim, hidden_size)), name='G_W1')
G_b1 = tf.Variable(tf.zeros(hidden_size), name='G_b1')

G_W2 = tf.Variable(xavier_init((hidden_size, mnist_shape)), name='G_W2')
G_b2 = tf.Variable(tf.zeros(mnist_shape), name='G_b2')


def generative_network(z):
    layer1 = tf.matmul(z, G_W1) + G_b1
    layer1 = tf.nn.relu(layer1)
    
    layer2 = tf.matmul(layer1, G_W2) + G_b2
    layer2 = tf.nn.sigmoid(layer2)

    return layer2


def noise_generator(shape):
    noise = np.random.uniform(-1., 1., size=shape)
    return noise

# Calculating net's outputs and losses

In [ ]:
fake_input = generative_network(input_G_ph)

D_real = discriminative_network(input_D_ph)
D_fake = discriminative_network(fake_input)

D_real_loss = tf.reduce_mean(tf.log(D_real))
D_fake_loss = tf.reduce_mean(tf.log(1. - D_fake))
D_loss = -(D_real_loss + D_fake_loss)

G_loss = -tf.reduce_mean(tf.log(D_fake))

# Training

In [ ]:
lr = 0.0001
D_train = tf.train.AdamOptimizer(learning_rate=lr).minimize(D_loss, var_list=[D_W1, D_b1, D_W2, D_b2])
G_train = tf.train.AdamOptimizer(learning_rate=lr).minimize(G_loss, var_list=[G_W1, G_b1, G_W2, G_b2])

In [ ]:
mnist_ex = input_data.read_data_sets("./data/", one_hot=True)

epoches = 1500
batch_size = 4096

config = tf.ConfigProto()
config.gpu_options.allow_growth = True
config.log_device_placement=True
sess = tf.Session()#config=config)

sess.run(tf.initialize_all_variables())
with sess:
    for e in range(epoches):
        d_losses, g_losses = [], []
        
        if e % 5 == 0:
            noise = noise_generator((9, z_dim))
            samples = sess.run(generative_network(input_G_ph), feed_dict={input_G_ph: noise})

            fig = plot(samples)
            plt.savefig('./out/{}.png'.format(str(e).zfill(3)), bbox_inches='tight')
            plt.close(fig)
            
        
        for step in range(mnist_ex.train.num_examples // batch_size):
            batchX, _ = mnist_ex.train.next_batch(batch_size)
            noise = noise_generator((batch_size, z_dim))
            
            d_loss, _ = sess.run([D_loss, D_train], feed_dict={input_D_ph: batchX, input_G_ph: noise})
            g_loss, _ = sess.run([G_loss, G_train], feed_dict={input_G_ph: noise})
            
            d_losses.append(d_loss)
            g_losses.append(g_loss)
        
        print("Epoch %d   D loss: %.5f   G loss: %.5f" % (e, d_loss, g_loss))